### FirstSparkETL

In [1]:
# Import required modules
from pyspark.sql import SparkSession
from datetime import datetime
from pyspark.sql import functions as f
from pyspark.sql.types import *
# Delta is a storage layer for data lakes
from delta.tables import * 
# DeltaTable is the main class for Delta tables
from delta.tables import DeltaTable 

# Initialize SparkSession
# Create a SparkSession and set the extraClassPath configuration
spark = SparkSession.builder.master("local[1]") \
    .appName("LetSparkWorkForYou") \
    .config("spark.driver.extraClassPath", "/home/jovyan/work/jars/*") \
    .getOrCreate()


# Define the schema
custom_schema = StructType([
    StructField("name", StringType(), True),
    StructField("roast", DoubleType(), True)
])

# Define the updated timestamp
UPDATED = datetime.today().replace(second=0)

# Initialize SparkSession
spark = SparkSession.builder.appName("coffeeETL").getOrCreate()

# Read the data with the specified schema and create a DataFrame
df = spark.read.format("csv") \
    .option("header", True) \
    .schema(custom_schema) \
    .load("/home/jovyan/work/data/raw-coffee.csv")

# Extract relevant columns and create a new DataFrame
new_df = df.select("name", "roast")

# Transform the data by adding a new column with current timestamp
transformed_df = new_df.withColumn('updated_at', f.lit(UPDATED))

# Create a new directory with timestamp suffix
timestamp = datetime.now().strftime("%Y%m%d")


output_dir = f"/home/jovyan/work/notebooks/cur_{timestamp}"

# Write the transformed data to a CSV file
transformed_df.write.option("header","true").csv(output_dir)

##########################################
# Stop the SparkSession
spark.stop()
##########################################

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
